In [2]:
import apache_beam as beam

# SQL
from apache_beam.transforms.sql import SqlTransform

# DataFrames
from apache_beam.dataframe.transforms import DataframeTransform
from apache_beam.dataframe.convert import to_dataframe
from apache_beam.dataframe.convert import to_pcollection

In [3]:
# taxirides data captured from projects/pubsub-public-data/topics/taxirides-realtime
!head -n2 taxirides.json | jq .

{
  "ride_id": "59d5d32d-c531-4e70-b50d-a1cd5ef0a747",
  "point_idx": 1759,
  "latitude": 40.727520000000005,
  "longitude": -73.88796,
  "timestamp": "2020-08-21T19:07:55.59142-04:00",
  "meter_reading": 43.6578,
  "meter_increment": 0.02481967,
  "ride_status": "enroute",
  "passenger_count": 1
}
{
  "ride_id": "7bf9bf5b-6d48-432c-bc2d-6b7de69096fb",
  "point_idx": 1424,
  "latitude": 40.73816,
  "longitude": -73.84964000000001,
  "timestamp": "2020-08-21T19:07:55.59139-04:00",
  "meter_reading": 31.309614,
  "meter_increment": 0.021987088,
  "ride_status": "enroute",
  "passenger_count": 1
}


In [3]:
#
# SqlTransform
#

In [4]:
import typing
import json

class TaxiPoint(typing.NamedTuple):
    ride_id: str
    point_idx: int
    latitude: float
    longitude: float
    timestamp: str
    meter_reading: float
    meter_increment: float
    ride_status: str
    passenger_count:  int
        
beam.coders.registry.register_coder(TaxiPoint, beam.coders.RowCoder)

In [5]:
with beam.Pipeline() as p:
    (p  | beam.io.ReadFromText("taxirides.json")
        | beam.Map(json.loads)
        | beam.Map(lambda x: TaxiPoint(**x)).with_output_types(TaxiPoint)
        
        
        | SqlTransform("""
                SELECT 
                  ride_status, 
                  COUNT(*) AS `count`
                FROM PCOLLECTION
                GROUP BY ride_status""")
        
        
        | beam.Map(lambda row: '%s:\t%d' % (row.ride_status, row.count))
        | beam.Map(print))

RuntimeError: json.decoder.JSONDecodeError: Unterminated string starting at: line 1 column 12 (char 11) [while running '[5]: Map(loads)']

In [6]:
with beam.Pipeline() as p:
    taxipoints = (
        p
        | beam.io.ReadFromText("taxirides.json")
        | beam.Map(json.loads)
        | beam.Map(lambda x: TaxiPoint(**x)).with_output_types(TaxiPoint))
    
    greetings = (p | beam.Create([
         ('pickup', 'On my way!'),
         ('dropoff', "I'm here!"),
       ]) | beam.Map(lambda x: beam.Row(ride_status=x[0], message=x[1])))
    
    
    ({'taxi': taxipoints, 'greetings': greetings} 
        | SqlTransform("""
                SELECT 
                  `timestamp`, 
                  taxi.ride_status, 
                  greetings.message AS greeting
                FROM taxi
                INNER JOIN greetings USING (ride_status)""")
     
     
        | beam.Map(lambda row: ', '.join(row))
        | beam.Map(print))

2020-08-21T19:07:54.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:54.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:54.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:54.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:56.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:56.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:56.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:56.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:56.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:56.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:56.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:55.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:55.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:55.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:55.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:55.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:55.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:55.19397-04:00, dropoff, I'm here!
2020-08-21T19:07:55.19397-04:00, dropoff, I'm here!
2020-08-21T1

In [7]:
#
# DataframeTransform
#

In [8]:
def explode_dataframe(df):
    for i in df.index:
        yield {column: df[column][i] for column in df.columns}
        
with beam.Pipeline() as p:
    taxipoints = (
        p
        | beam.io.ReadFromText("taxirides.json")
        | beam.Map(json.loads)
        | beam.Map(lambda x: TaxiPoint(**x)).with_output_types(TaxiPoint))
    
    
    df = to_dataframe(taxipoints)
    
    filtered_df = df.loc[(df.passenger_count > 1) & (df.ride_status == 'dropoff')]
    result = filtered_df[['timestamp', 'ride_status', 'passenger_count']]
    
    
    (to_pcollection(result) 
        | beam.FlatMap(explode_dataframe)
        | beam.Map(print))

{'timestamp': '2020-08-21T19:07:55.19397-04:00', 'ride_status': 'dropoff', 'passenger_count': 2}
{'timestamp': '2020-08-21T19:06:24.19397-04:00', 'ride_status': 'dropoff', 'passenger_count': 2}
{'timestamp': '2020-08-21T19:07:57.19397-04:00', 'ride_status': 'dropoff', 'passenger_count': 4}
{'timestamp': '2020-08-21T19:07:56.19397-04:00', 'ride_status': 'dropoff', 'passenger_count': 3}
{'timestamp': '2020-08-21T19:07:57.19397-04:00', 'ride_status': 'dropoff', 'passenger_count': 5}
{'timestamp': '2020-08-21T19:07:58.19397-04:00', 'ride_status': 'dropoff', 'passenger_count': 5}
{'timestamp': '2020-08-21T19:07:58.19397-04:00', 'ride_status': 'dropoff', 'passenger_count': 4}
{'timestamp': '2020-08-21T19:07:54.19397-04:00', 'ride_status': 'dropoff', 'passenger_count': 3}
{'timestamp': '2020-08-21T19:07:55.19397-04:00', 'ride_status': 'dropoff', 'passenger_count': 6}
{'timestamp': '2020-08-21T19:07:56.19397-04:00', 'ride_status': 'dropoff', 'passenger_count': 2}
{'timestamp': '2020-08-21T19:0

In [9]:
#
# GroupBy
#

In [10]:
with beam.Pipeline() as p:
  (p | beam.io.ReadFromText("taxirides.json")
     | beam.Map(json.loads)
     | beam.Map(lambda x: TaxiPoint(**x)).with_output_types(TaxiPoint)
   
   
     # Compute meter increment and bounding box for each ride
     | beam.GroupBy('ride_id')
       .aggregate_field('meter_increment', sum, 'meter_sum')
       .aggregate_field('latitude', min, 'min_lat')
       .aggregate_field('latitude', max, 'max_lat')
       .aggregate_field('longitude', min, 'min_lng')
       .aggregate_field('longitude', max, 'max_lng')
   
   
     | beam.Map(lambda row: '{0.ride_id}: meter_sum={0.meter_sum:.3f}, bbox={0.min_lat: .3f},{0.min_lng: .3f} x {0.max_lat: .3f},{0.max_lng: .3f}'.format(row))
     | beam.Map(print))

59d5d32d-c531-4e70-b50d-a1cd5ef0a747: meter_sum=0.223, bbox= 40.726,-73.901 x  40.728,-73.888
7bf9bf5b-6d48-432c-bc2d-6b7de69096fb: meter_sum=0.132, bbox= 40.738,-73.850 x  40.738,-73.850
e049e5b2-375b-4f56-9437-131dd4de30e7: meter_sum=0.098, bbox= 40.710,-73.992 x  40.711,-73.984
2ee3f1a9-a864-40de-85ff-d16732101715: meter_sum=0.146, bbox= 40.790,-73.982 x  40.799,-73.975
ab84a484-88ae-4437-8d1f-ec98deec559d: meter_sum=0.120, bbox= 40.687,-73.976 x  40.688,-73.971
93f4807d-bc99-4c61-953a-6dbebbf4dea2: meter_sum=0.176, bbox= 40.667,-73.740 x  40.678,-73.730
349b8b71-f5aa-492f-b9d7-054b68cfaf1a: meter_sum=0.091, bbox= 40.754,-74.003 x  40.758,-74.001
173c5ebd-6270-490c-89ce-d665453ee31d: meter_sum=0.099, bbox= 40.734,-73.991 x  40.735,-73.988
d1cc2398-f3f6-4ed6-ab5d-80c9dfbf3509: meter_sum=0.134, bbox= 40.767,-73.963 x  40.771,-73.960
212f08fd-8962-4f3b-a428-62db5dcfdfb9: meter_sum=0.110, bbox= 40.774,-73.993 x  40.780,-73.988
c77a5d00-4ac0-487f-b147-7f84b628de24: meter_sum=0.103, bbox=